# COURSERA CAPSTONE PROJECT

### This notebook will be used primarily for the capstone project

In [1]:
# Importing packages
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


# BATTLE OF THE NEIGHBORHOODS - OAK PARK VS. EVANSTON, IL

### INTRODUCTION & BUSINESS PROBLEM

In this notebook we will analyze two afluent suburbs of Chicago, IL.

The suburbs are Evanston and Oak Park. 

Evanston, which is 12 miles (19 km) north of Chicago, is known for being home to Northwestern University, a well-known college.<sup>1</sup>

Oak Park, on the other hand, sits 10 miles (16 km) west of Chicago, and it is known for architect Frank Lloyd Wright and writer Ernest Hemingway. <sup>2</sup>

Both places are the most common consideration for couples with kids looking to move out of Chicago for more space while retaining easy access to the city. 

Evanston and Oak Park have good schools, and similar real estate markets. Access to Chicago is easy by using both train or the subway system, in addition to interstates for car access. 

This project aims to help couples make an informed decision based on their personal preferences. 

<sup>1</sup> https://en.wikipedia.org/wiki/Evanston,_Illinois

<sup>2</sup> https://en.wikipedia.org/wiki/Oak_Park,_Illinois

### DATA REQUIREMENTS


For this study, we will leverage data from Python library Geocode & Foursquare to compare Evanston and Oak Park based on a variety of options including restaurants, entertainment, and public interest places. Here's how I will get & process the data for this notebook: 

1) From Geocode I will create a querry to get the latitude and longitude of both Evanston and Oak Park

2) I will then utilize the Foursquare API to fetch data on all the venues and point of interest within 1km of the latitude and longitude acquired in step # 1

3) I will process and clean-up the json file in order to convert to a dataframe and use with the pandas library

4) With the dataframe we will use one hot encoding and classify each venue type by frequencies for both neighborhoods. This should make comparison straighforward

5) Finally we will leverage pandas and other python tools to further deepen our analysis and compare how similar or dissimilar both neighborhoods are

 

### OAK PARK VS. EVANSTON

### Let's import most packages we need

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

!conda install -c conda-forge geopy --yes
import geopy

from geopy.geocoders import Nominatim

import json

from pandas.io.json import json_normalize

import matplotlib.cm as cm

import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('All libraries imported!')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

All libraries imported!


### Let's get coordinates for both Oak Park and Evanston

In [4]:

address1 = 'Oak Park, Illinois'
address2 = 'Evanston, Illinois'

geolocator = Nominatim(user_agent="opark_evanston_explorer")
location1 = geolocator.geocode(address1)
location2 = geolocator.geocode(address2)
latitude1 = location1.latitude
longitude1 = location1.longitude
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of Oak Park are {}, {}.'.format(latitude1, longitude1))
print('The geograpical coordinate of Evanston are {}, {}.'.format(latitude2, longitude2))

The geograpical coordinate of Oak Park are 41.8878145, -87.7887615.
The geograpical coordinate of Evanston are 42.0447388, -87.6930459.


### It's Map Time!!!

In [5]:
map_oakpark = folium.Map(location=[latitude1, longitude1], zoom_start=14)

map_oakpark

In [6]:
map_evanston = folium.Map(location =[latitude2, longitude2], zoom_start=14)

map_evanston

### Foursquare data

In [7]:
# Foursquare Credentials

CLIENT_ID = 'KLOZOTEMC3WB0RUI20IRL4WBSFBPS4QVY0ZCKLH5BFN3UOI0' # Foursquare ID
CLIENT_SECRET = 'GNC3U1DSZZX25FJ2KWPPUVTSZJ1RQQ3UWHQ3FKXGSGVXGFN1' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

radius = 1000

LIMIT = 200

# We will start with OAK PARK

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)
            
import requests

results = requests.get(url).json()

items = results['response']['groups'][0]['items']
items[1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4db882bcfa8c978590ba9e26',
  'name': 'Ahimsa Yoga Studio',
  'location': {'address': '441 South Blvd',
   'lat': 41.88691743815367,
   'lng': -87.78682992439704,
   'labeledLatLngs': [{'label': 'display',
     'lat': 41.88691743815367,
     'lng': -87.78682992439704}],
   'distance': 188,
   'postalCode': '60302',
   'cc': 'US',
   'city': 'Oak Park',
   'state': 'IL',
   'country': 'United States',
   'formattedAddress': ['441 South Blvd',
    'Oak Park, IL 60302',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d102941735',
    'name': 'Yoga Studio',
    'pluralName': 'Yoga Studios',
    'shortName': 'Yoga Studio',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []},
  'venuePage': {'id': '59

In [8]:
# Create DataFrame

oakpark_df = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name','venue.location.address','venue.location.city',
                    'venue.location.state','venue.location.postalCode','venue.location.lat',
                    'venue.location.lng','venue.categories']

oakpark_filtered = oakpark_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
oakpark_filtered['categories'] = oakpark_filtered.apply(get_category_type, axis=1)

del oakpark_filtered['venue.categories']

oakpark_filtered.rename(columns={'venue.name': 'venue_name', 'venue.location.address': 'venue_address', 
                                'venue.location.city': 'venue_city','venue.location.state': 'venue_state',
                                'venue.location.postalCode': 'venue_postalCode',
                                'venue.location.lat':'venue_latitude', 'venue.location.lng': 'venue_longitude',
                                'categories': 'venue_categorie'},
                       inplace = True)

oakpark_filtered.shape

(50, 8)

### We can see that Oak Park returned 51 different venues from Foursquare

### Let's check the most common venues 

In [9]:
# Let's use one hot encoding

oakpark_hot = pd.get_dummies(oakpark_filtered[['venue_categorie']], prefix="", prefix_sep="")

# add back Oak Park as column

oakpark_hot['Neighborhood'] = 'Oak Park'

oakpark_hot = oakpark_hot[ ['Neighborhood'] + [ col for col in oakpark_hot.columns if col != 'Neighborhood' ] ]

oakpark_hot = oakpark_hot.groupby('Neighborhood').mean().reset_index()

# Check top 10 venues

num_top_venues = 10

for hood in oakpark_hot['Neighborhood']:
    print("----"+hood+"----")
    temp = oakpark_hot[oakpark_hot['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Oak Park----
                 venue  freq
0   Italian Restaurant  0.06
1     Toy / Game Store  0.06
2        Historic Site  0.06
3               Bakery  0.04
4  American Restaurant  0.04
5          Coffee Shop  0.04
6                  Gym  0.04
7                  ATM  0.02
8             Pharmacy  0.02
9            Multiplex  0.02




### Let's repeat the same process but for Evanston

In [10]:
# Foursquare Credentials

CLIENT_ID = 'KLOZOTEMC3WB0RUI20IRL4WBSFBPS4QVY0ZCKLH5BFN3UOI0' # Foursquare ID
CLIENT_SECRET = 'GNC3U1DSZZX25FJ2KWPPUVTSZJ1RQQ3UWHQ3FKXGSGVXGFN1' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

radius = 1000

LIMIT = 200

# Now for Evanston

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)
            
import requests

results = requests.get(url).json()

items = results['response']['groups'][0]['items']
items[1]



{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57c9b7a5498e5242e66d75dc',
  'name': 'Taco Diablo',
  'location': {'address': '1026 Davis St',
   'lat': 42.04693190710444,
   'lng': -87.68620413708848,
   'labeledLatLngs': [{'label': 'display',
     'lat': 42.04693190710444,
     'lng': -87.68620413708848}],
   'distance': 616,
   'postalCode': '60201',
   'cc': 'US',
   'city': 'Evanston',
   'state': 'IL',
   'country': 'United States',
   'formattedAddress': ['1026 Davis St',
    'Evanston, IL 60201',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '444247',
   'url': 'https://www.grubhub.com/restaurant/taco-diablo-1026

In [11]:
# Create DataFrame

evanston_df = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name','venue.location.address','venue.location.city',
                    'venue.location.state','venue.location.postalCode','venue.location.lat',
                    'venue.location.lng','venue.categories']

evanston_filtered = evanston_df.loc[:, filtered_columns]

# filter the category for each row - we already defined the get category previously
evanston_filtered['categories'] = evanston_filtered.apply(get_category_type, axis=1)

del evanston_filtered['venue.categories']

evanston_filtered.rename(columns={'venue.name': 'venue_name', 'venue.location.address': 'venue_address', 
                                'venue.location.city': 'venue_city','venue.location.state': 'venue_state',
                                'venue.location.postalCode': 'venue_postalCode',
                                'venue.location.lat':'venue_latitude', 'venue.location.lng': 'venue_longitude',
                                'categories': 'venue_categorie'},
                       inplace = True)

evanston_filtered.shape

(91, 8)

### We can see that Evanston returned 87 different venues from Foursquare

### Let's check the most common venues

In [12]:
# Let's use one hot encoding

evanston_hot = pd.get_dummies(evanston_filtered[['venue_categorie']], prefix="", prefix_sep="")

# add back Evanston as column

evanston_hot['Neighborhood'] = 'Evanston'

evanston_hot = evanston_hot[ ['Neighborhood'] + [ col for col in evanston_hot.columns if col != 'Neighborhood' ] ]

evanston_hot = evanston_hot.groupby('Neighborhood').mean().reset_index()

# Check top 10 venues

num_top_venues = 10

for hood in evanston_hot['Neighborhood']:
    print("----"+hood+"----")
    temp = evanston_hot[evanston_hot['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Evanston----
                  venue  freq
0           Coffee Shop  0.05
1       Thai Restaurant  0.04
2   Rental Car Location  0.03
3  Gym / Fitness Center  0.03
4          Burger Joint  0.03
5        Clothing Store  0.02
6                  Café  0.02
7               Brewery  0.02
8            Playground  0.02
9                   Gym  0.02




### Let's try to compare Oak Park and Evanston

In [13]:
# Let's concatenate both dataframes

df = pd.concat([oakpark_hot, evanston_hot])
df = df.reset_index(drop = True)

# Group by columns

df_grouped = df.groupby(list(df.columns))

# Index with categories

idx = [x[0] for x in df_grouped.groups.values() if len(x) == 1]

df.reindex(idx)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,ATM,American Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Bike Shop,Bookstore,Breakfast Spot,Brewery,Burger Joint,Café,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Discount Store,Donut Shop,Farmers Market,Fast Food Restaurant,Food Stand,Food Truck,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Italian Restaurant,Jewelry Store,Latin American Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Multiplex,Music Store,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Salon / Barbershop,Sandwich Place,Shopping Mall,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Storage Facility,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0.02,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,0.040000,0.02,0.020000,NaN,NaN,0.020000,0.020000,0.020000,NaN,0.020000,NaN,NaN,0.040000,0.020000,NaN,NaN,0.020000,NaN,NaN,0.02,0.02,NaN,NaN,NaN,NaN,NaN,0.020000,0.02,0.040000,0.020000,NaN,0.06,0.02,NaN,0.020000,0.060000,NaN,0.02,NaN,0.02,0.02,NaN,NaN,NaN,NaN,0.02,NaN,0.02,Oak Park,0.02,NaN,NaN,0.02,NaN,0.02,0.020000,NaN,NaN,NaN,0.02,NaN,0.02,NaN,0.020000,NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN,0.06,0.02,0.02,NaN,0.02
1,NaN,0.010989,0.010989,0.010989,0.010989,0.010989,0.010989,0.010989,0.021978,NaN,0.021978,0.010989,0.010989,0.010989,0.010989,0.021978,0.032967,0.021978,0.021978,0.010989,0.054945,0.010989,0.010989,0.021978,0.010989,0.010989,0.010989,NaN,NaN,0.010989,0.010989,0.010989,0.010989,0.010989,0.010989,NaN,0.021978,0.032967,0.010989,NaN,NaN,0.010989,0.010989,0.021978,0.010989,NaN,0.010989,NaN,NaN,0.010989,0.010989,0.010989,0.010989,NaN,0.010989,NaN,Evanston,NaN,0.021978,0.010989,NaN,0.010989,NaN,0.010989,0.021978,0.010989,0.010989,NaN,0.032967,NaN,0.010989,0.021978,0.010989,NaN,0.010989,0.010989,0.010989,0.010989,0.010989,0.010989,0.010989,NaN,0.010989,0.010989,0.043956,0.010989,NaN,NaN,NaN,0.010989,NaN


In [14]:
# Let's transpose the dataframe

tp_df = df.T

tp_df.rename(columns = {0:'Oak Park', 1:'Evanston'}, inplace = True)

tp_df

,Oak Park,Evanston
ATM,0.02,NaN
American Restaurant,0.04,0.010989
Antique Shop,NaN,0.010989
Arts & Crafts Store,NaN,0.010989
Asian Restaurant,NaN,0.010989
Athletics & Sports,NaN,0.010989
Automotive Shop,NaN,0.010989
BBQ Joint,NaN,0.010989
Bakery,0.04,0.021978
Bank,0.02,NaN


In [15]:
# Let's create the final dataframe showing index with categories and which neighborhood has each

tp_df.loc[tp_df['Oak Park'].isna(), 'Comparison'] = 'Unique to Evanston'

tp_df.loc[tp_df['Evanston'].isna(), 'Comparison'] = 'Unique to Oak Park'

tp_df['Oak Park'] = pd.to_numeric(tp_df['Oak Park'], errors='coerce')

tp_df['Evanston'] = pd.to_numeric(tp_df['Evanston'], errors='coerce')

tp_df.loc[tp_df['Comparison'].isna(), 'Comparison'] = tp_df['Oak Park'] - tp_df['Evanston']

tp_df

,Oak Park,Evanston,Comparison
ATM,0.02,NaN,Unique to Oak Park
American Restaurant,0.04,0.010989,0.029011
Antique Shop,NaN,0.010989,Unique to Evanston
Arts & Crafts Store,NaN,0.010989,Unique to Evanston
Asian Restaurant,NaN,0.010989,Unique to Evanston
Athletics & Sports,NaN,0.010989,Unique to Evanston
Automotive Shop,NaN,0.010989,Unique to Evanston
BBQ Joint,NaN,0.010989,Unique to Evanston
Bakery,0.04,0.021978,0.018022
Bank,0.02,NaN,Unique to Oak Park


In [16]:
# Let's see what is unique to Oak Park

op_df = tp_df['Comparison'] == 'Unique to Oak Park'

op_df = op_df.astype(int)

op_exclusive = op_df[op_df.loc[:]!=0].dropna()

op_exclusive

ATM                                         1
Bank                                        1
Food Truck                                  1
French Restaurant                           1
Grocery Store                               1
Historic Site                               1
Hobby Shop                                  1
Latin American Restaurant                   1
Lounge                                      1
Mediterranean Restaurant                    1
Multiplex                                   1
Nail Salon                                  1
New American Restaurant                     1
Park                                        1
Pharmacy                                    1
Record Shop                                 1
Residential Building (Apartment / Condo)    1
Southern / Soul Food Restaurant             1
Tapas Restaurant                            1
Toy / Game Store                            1
Train Station                               1
Vietnamese Restaurant             

In [17]:
# Now let's see what is unique to Evanston

et_df = tp_df['Comparison'] == 'Unique to Evanston'

et_df = et_df.astype(int)

et_exclusive = et_df[et_df.loc[:]!=0].dropna()

et_exclusive

Antique Shop                     1
Arts & Crafts Store              1
Asian Restaurant                 1
Athletics & Sports               1
Automotive Shop                  1
BBQ Joint                        1
Bed & Breakfast                  1
Bike Shop                        1
Burger Joint                     1
Clothing Store                   1
Cocktail Bar                     1
Discount Store                   1
Donut Shop                       1
Fast Food Restaurant             1
Food Stand                       1
Fruit & Vegetable Store          1
Furniture / Home Store           1
Garden                           1
Gastropub                        1
Government Building              1
Gym Pool                         1
Hotel                            1
Jewelry Store                    1
Liquor Store                     1
Metro Station                    1
Mexican Restaurant               1
Mobile Phone Shop                1
Movie Theater                    1
Music Store         

In [35]:
# Finally let's see what both neighborhoods offer
both_df = tp_df.drop(tp_df[tp_df['Comparison'] == 'Unique to Oak Park'].index)

both_df = both_df.drop(both_df[both_df['Comparison'] == 'Unique to Evanston'].index)

both_df

,Oak Park,Evanston,Comparison
American Restaurant,0.04,0.010989,0.029011
Bakery,0.04,0.021978,0.018022
Bar,0.02,0.021978,-0.00197802
Bookstore,0.02,0.010989,0.00901099
Breakfast Spot,0.02,0.010989,0.00901099
Brewery,0.02,0.021978,-0.00197802
Café,0.02,0.021978,-0.00197802
Coffee Shop,0.04,0.054945,-0.0149451
Convenience Store,0.02,0.010989,0.00901099
Farmers Market,0.02,0.010989,0.00901099


# This concludes this notebook. Thank you.